In [1]:
# Imports
import os
import bz2
import numpy as np
from PIL import TiffImagePlugin
from IPython.core.display import display
from PIL import Image as Img
import sys
sys.path.append("../")
from algorithms.flow import flow
from data_structures.location_graph import LocationGraph
from utils.dummy_writer import DummyWriter

In [3]:
def load_image_data() -> TiffImagePlugin.TiffImageFile:
    path = os.path.dirname(os.path.abspath('')) + '/data/'
    data = bz2.decompress(open(path + 'dem.tif.bz2', 'rb').read())
    with open(path + 'dem2.tiff', 'wb') as handle:
        handle.write(data)
    return Img.open(path + 'dem2.tiff')

In [4]:
def load_and_crop_data(crop_size, offset):
    raw_height_array = np.asarray(load_image_data())
    cropped = raw_height_array[offset:offset+crop_size, offset:offset+crop_size]
    print('Loaded height data')
    return cropped

In [5]:
def normalize_values(i):
    return int((i-1)*255/1193)

def get_colour_function(flows):
    return normalize_values

In [6]:
class ImageWriter(object):
    def __init__(self, path, size, save_frequency):
        if not os.path.exists(path):
            os.makedirs(path)
        self.write_path = path
        self.index = 0
        self.image = Img.new('L', (size, size))
        self.pixels = self.image.load()
        self.save_frequency=save_frequency
        self.image_number = 0
        self.size = size
        self.previous_percent = 0

    def update(self, node):
        self.index += 1
        self.pixels[node.home[0],node.home[1]] = normalize_values(node.flow)
        if self.index % self.save_frequency == 0:
            filename = f"{self.write_path}{self.image_number}.tiff"
            (self.image).save(filename)
            
            percent_finished = int(self.index*100/self.size**2)
            if percent_finished != self.previous_percent and percent_finished%2==0:
                print(f'Processed {percent_finished}% of {self.size**2} images.')
                self.previous_percent = percent_finished
            self.image_number += 1

# Initialize image file writer
def initialize_writer(size, save_frequency, job_name):
    writer_path = f"{os.path.dirname(os.path.dirname(os.path.abspath('')))}/data/{job_name}/"
    print(f"Writer will save images to {writer_path}")
    writer = ImageWriter(writer_path, size, save_frequency)
    return writer

In [7]:
# Do a run with no images
# Configure colour function
# Do a run generating images
size = 3601
frequency = 10000
job_name = "fullscale"

def report_settings():
    num_images = int(size**2/frequency)
    print(f"Num images: {num_images}")
    total_size = ((size/1000)**2)*num_images
    if total_size > 1000:
        print(f"Total data size: {total_size/1000.0:.2f} GB")
    else:
        print(f"Total data size: {total_size:.2f} MB")
        
report_settings()
writer = initialize_writer(size, frequency, job_name)

Num images: 1296
Total data size: 16.81 GB
Writer will save images to /home/andrew/personal/river-flow/data/fullscale/


In [8]:
%%time
height_map = load_and_crop_data(size,0).tolist()
print("Creating Graph")
graph = LocationGraph(height_map)

Loaded height data
Creating Graph
CPU times: user 7min 17s, sys: 39.4 s, total: 7min 56s
Wall time: 7min 56s


In [9]:
%%time
print("Running flow for colour calibration")
nodes_with_flow = flow(graph, DummyWriter())
flows = [n.flow for n in nodes_with_flow.ascending()]
colour_function = get_colour_function(flows)

Running flow for colour calibration
CPU times: user 53.2 s, sys: 171 ms, total: 53.4 s
Wall time: 53.4 s


In [10]:
%%time
print("Running real flow simulation")
flow(graph, writer)

Running real flow simulation
Processed 2% of 12967201 images.
Processed 4% of 12967201 images.
Processed 6% of 12967201 images.
Processed 8% of 12967201 images.
Processed 10% of 12967201 images.
Processed 12% of 12967201 images.
Processed 14% of 12967201 images.
Processed 16% of 12967201 images.
Processed 18% of 12967201 images.
Processed 20% of 12967201 images.
Processed 22% of 12967201 images.
Processed 24% of 12967201 images.
Processed 26% of 12967201 images.
Processed 28% of 12967201 images.
Processed 30% of 12967201 images.
Processed 32% of 12967201 images.
Processed 34% of 12967201 images.
Processed 36% of 12967201 images.
Processed 38% of 12967201 images.
Processed 40% of 12967201 images.
Processed 42% of 12967201 images.
Processed 44% of 12967201 images.
Processed 46% of 12967201 images.
Processed 48% of 12967201 images.
Processed 50% of 12967201 images.
Processed 52% of 12967201 images.
Processed 54% of 12967201 images.
Processed 56% of 12967201 images.
Processed 58% of 129672